In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, InputLayer, Input, add, dot, maximum, average, multiply, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split

In [2]:
# reading data
data = pd.read_csv('PrOCTOR_sample_data_all.csv', header=0)
data1 = data.fillna(data.mean()['MolecularWeight':'Salivary Gland'])
data1["target"] = np.where(data1.iloc[:, 1] == "passed", 1, 0)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# x, y variable
X = data1.iloc[:, 2:-1]
y = data1['target']
#print(X)

In [4]:
X = np.array(X)
X = scaler.fit_transform(X)
y = np.array(y)
y_1 = y.reshape(y.shape[0],-1)
#y = scaler.fit_transform(X)
des=X[:,:13]
body=X[:,13:]

In [5]:
print(des.shape)
print(body.shape)
print(y.shape)
print(y_1.shape)

(828, 13)
(828, 34)
(828,)
(828, 1)


In [6]:
hid_layer = [100, 100, 50]
print(hid_layer[-1])

50


In [7]:
_, des_col = des.shape
_, body_col = body.shape

In [8]:
class MM_Sum:
    def __init__(self, des_shape, body_shape, hid_layer, output_layer):
        self.des_shape = des_shape
        self.body_shape = body_shape
        self.hid_layer = hid_layer  # not used
        self.output_layer = output_layer
        
    def Multi_modal_Sum(self):
        # des modality
        des_input = Input(shape = self.des_shape) # input = 17, des_shape = 17
        des_input_x2 = Dense(self.hid_layer[0], activation = 'relu')(des_input)
        des_input_x2_batch = BatchNormalization()(des_input_x2)
        des_input_x3 = Dense(self.hid_layer[1], activation = 'relu')(des_input_x2_batch)
        des_input_x3_batch = BatchNormalization()(des_input_x3)
        des_input_x4 = Dense(self.hid_layer[2], activation = 'relu')(des_input_x3_batch)
        des_input_x4_batch = BatchNormalization()(des_input_x4)
        
        # body modality
        body_input = Input(shape = self.body_shape) # input = 30,body_shape = 30
        body_input_x2 = Dense(self.hid_layer[0], activation = 'relu')(body_input)
        body_input_x2_batch = BatchNormalization()(body_input_x2)
        body_input_x3 = Dense(self.hid_layer[1], activation = 'relu')(body_input_x2_batch)
        body_input_x3_batch = BatchNormalization()(body_input_x3)
        body_input_x4 = Dense(self.hid_layer[2], activation = 'relu')(body_input_x3_batch)
        body_input_x4_batch = BatchNormalization()(body_input_x4)
        
        sum_layer = keras.layers.add([des_input_x4_batch, body_input_x4_batch])
        sum_input_x = Dense(self.output_layer, activation = 'sigmoid')(sum_layer)#(des_input_flat)
        sum_model = keras.models.Model(inputs = [des_input, body_input], outputs = [sum_input_x])
        
        multiply_layer = multiply([des_input_x4_batch, body_input_x4_batch])       
        multiply_input_x = Dense(self.output_layer, activation = 'sigmoid')(multiply_layer)
        multiply_model = keras.models.Model(inputs = [des_input, body_input], outputs = [multiply_input_x])
        
        concat_layer = concatenate([des_input_x4_batch, body_input_x4_batch])        
        concat_input_x = Dense(self.output_layer, activation = 'sigmoid')(concat_layer)
        concat_model = keras.models.Model(inputs = [des_input, body_input], outputs = [concat_input_x])
        

        return sum_model, multiply_model, concat_model

    

In [9]:
final_model = MM_Sum(des_col, body_col, hid_layer, 1) # input type = integer
network = final_model.Multi_modal_Sum()
print(network)

(<tensorflow.python.keras.engine.functional.Functional object at 0x0000022551443C08>, <tensorflow.python.keras.engine.functional.Functional object at 0x0000022551429088>, <tensorflow.python.keras.engine.functional.Functional object at 0x00000225513D6AC8>)


In [10]:
sum_net = network[0]
multiply_net = network[1]
concat_net = network[2]
#dot_net = network[2]

In [11]:
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

# 모델설정
sm = SMOTE(random_state=202004)
loo = LeaveOneOut()
kfold = KFold(n_splits = 10, shuffle = True, random_state = 111)
kfold.get_n_splits(X)

10

In [12]:
def predict_change(predict):
    y_predict = []
    for i in range(len(predict)):
        y_predict = np.append(y_predict, predict[i])
    
    y_predict = np.array(y_predict)
    y_predict = y_predict.reshape(y_predict.shape[0],-1)
    return y_predict

In [13]:
dict_y = dict()

class train_model:
    def __init__(self, x, y, model, pd_y, pd_sm_y, pd_cw_y, pd_smcw_y):
        self.X = x
        self.y = y
        self.model = model
        self.pd_y = pd_y
        self.pd_sm_y = pd_sm_y
        self.pd_cw_y = pd_cw_y
        self.pd_smcw_y = pd_smcw_y
        
    def train_based(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            self.model.compile(optimizer= keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [des_train, body_train], y = y_train, epochs=50)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
                
        return self.pd_y
    
    def train_smote(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)

            self.model.compile(optimizer= keras.optimizers.Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [sm_des_train, sm_body_train], y = sm_y_train, epochs=50)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_sm_y[index] = y_pred[i]
            
        return self.pd_sm_y
    
    def train_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
            
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [des_train, body_train], y = y_train, epochs=50, class_weight = class_weight)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_cw_y[index] = y_pred[i]
            
        return self.pd_cw_y
    
    def train_sm_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
    
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)
    
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [sm_des_train, sm_body_train], y = sm_y_train, epochs=50, class_weight = class_weight)

            y_pred = self.model.predict([des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_smcw_y[index] = y_pred[i]
            
        return self.pd_smcw_y
    

In [98]:
final_model = MM_Sum(des_col, body_col, hid_layer, 1) # input type = integer
network = final_model.Multi_modal_Sum()
print(network)
sum_net = network[0]

sum_y, sum_sm_y, sum_cw_y, sum_smcw_y = [], [], [], []
output_sum = train_model(X, y, sum_net, dict_y, dict_y, dict_y, dict_y)
# sum model training
sum_y = output_sum.train_based()
#sum_sm_y = output_sum.train_smote()
#sum_cw_y = output_sum.train_weight()
#sum_smcw_y = output_sum.train_sm_weight()

(<tensorflow.python.keras.engine.training.Model object at 0x0000025CA1D56788>, <tensorflow.python.keras.engine.training.Model object at 0x0000025CA67A2688>, <tensorflow.python.keras.engine.training.Model object at 0x0000025C98C62688>)
TEST: [  1   5  11  12  34  36  39  46  49  52  56  68  71  93 102 104 112 119
 122 125 126 140 145 152 157 162 164 166 167 183 190 194 198 205 209 210
 211 214 223 228 230 233 234 237 249 257 258 264 272 277 280 281 291 298
 303 323 326 328 331 332 338 340 342 354 366 368 373 374 375 380 381 383
 389 399 401 403 409 412 413 421 422 423 431 432 434 443 444 447 456 460
 461 464 465 468 471 474 476 481 491 493 496 498 506 507 516 518 520 523
 526 528 532 537 538 547 551 553 556 572 575 581 586 589 604 606 611 613
 624 629 632 645 651 652 653 658 664 665 666 673 694 700 701 702 706 707
 708 709 718 723 735 744 745 754 757 766 767 769 774 780 783 789 799 800
 806 816 822 825]
Epoch 1/50
21/21 [==============================] - 0s 2ms/step - loss: 0.7803 - acc

21/21 [==============================] - 0s 856us/step - loss: 0.0260 - accuracy: 0.9909
Epoch 42/50
21/21 [==============================] - 0s 841us/step - loss: 0.0261 - accuracy: 0.9894
Epoch 43/50
21/21 [==============================] - 0s 854us/step - loss: 0.0288 - accuracy: 0.9894
Epoch 44/50
21/21 [==============================] - 0s 181us/step - loss: 0.0220 - accuracy: 0.9909
Epoch 45/50
21/21 [==============================] - 0s 866us/step - loss: 0.0226 - accuracy: 0.9955
Epoch 46/50
21/21 [==============================] - 0s 855us/step - loss: 0.0123 - accuracy: 0.9970
Epoch 47/50
21/21 [==============================] - 0s 902us/step - loss: 0.0153 - accuracy: 0.9909
Epoch 48/50
21/21 [==============================] - 0s 901us/step - loss: 0.0103 - accuracy: 0.9955
Epoch 49/50
21/21 [==============================] - 0s 855us/step - loss: 0.0118 - accuracy: 0.9940
Epoch 50/50
21/21 [==============================] - 0s 648us/step - loss: 0.0139 - accuracy: 0.9970
TE

In [120]:
final_model = MM_Sum(des_col, body_col, hid_layer, 1) # input type = integer
network = final_model.Multi_modal_Sum()
print(network)
multiply_net = network[1]

multiply_y, multiply_sm_y, multiply_cw_y, multiply_smcw_y =[], [], [], []
output_mul = train_model(X, y, multiply_net, dict_y, dict_y, dict_y, dict_y)

# multiple model training
multiply_y = output_mul.train_based()
#multiply_sm_y = output_mul.train_smote()
#multiply_cw_y = output_mul.train_weight()
#multiply_smcw_y = output_mul.train_sm_weight()

(<tensorflow.python.keras.engine.training.Model object at 0x0000023B7BA8F588>, <tensorflow.python.keras.engine.training.Model object at 0x0000023B7BAA6508>, <tensorflow.python.keras.engine.training.Model object at 0x0000023B7BA930C8>)
TEST: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82]
Epoch 1/50
43/43 [==============================] - 0s 1ms/step - loss: 1.4099 - accuracy: 0.6718
Epoch 2/50
43/43 [==============================] - 0s 1ms/step - loss: 0.7520 - accuracy: 0.7478
Epoch 3/50
43/43 [==============================] - 0s 1ms/step - loss: 0.5378 - accuracy: 0.7729
Epoch 4/50
43/43 [==============================] - 0s 1ms/step - loss: 0.4686 - accuracy: 0.7920
Epoch 5/50
43/43 [==============================] - 0s 1ms/step - loss: 0.3623 - accuracy: 0.8422
Epoch 6/50
43/4

Epoch 1/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0998 - accuracy: 0.9809
Epoch 2/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0933 - accuracy: 0.9757
Epoch 3/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0459 - accuracy: 0.9786
Epoch 4/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0366 - accuracy: 0.9860
Epoch 5/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0290 - accuracy: 0.9867
Epoch 6/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0401 - accuracy: 0.9860
Epoch 7/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0327 - accuracy: 0.9897
Epoch 8/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0278 - accuracy: 0.9941
Epoch 9/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0348 - accuracy: 0.9882
Epoch 10/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0538 - accuracy: 0.9772
Epoch 11/50
43/43 [

43/43 [==============================] - 0s 1ms/step - loss: 0.0360 - accuracy: 0.9860
Epoch 31/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0361 - accuracy: 0.9875
Epoch 32/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0311 - accuracy: 0.9890
Epoch 33/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0523 - accuracy: 0.9905
Epoch 34/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0594 - accuracy: 0.9765
Epoch 35/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0374 - accuracy: 0.9860
Epoch 36/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0268 - accuracy: 0.9890
Epoch 37/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0428 - accuracy: 0.9846
Epoch 38/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0726 - accuracy: 0.9802
Epoch 39/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0544 - accuracy: 0.9824
Epoch 40/50
43/43 [===

43/43 [==============================] - 0s 2ms/step - loss: 0.0400 - accuracy: 0.9860
Epoch 7/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0232 - accuracy: 0.9919
Epoch 8/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0283 - accuracy: 0.9874
Epoch 9/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0245 - accuracy: 0.9897
Epoch 10/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0118 - accuracy: 0.9926
Epoch 11/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0100 - accuracy: 0.9970
Epoch 12/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0425 - accuracy: 0.9889
Epoch 13/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0164 - accuracy: 0.9934
Epoch 14/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0179 - accuracy: 0.9934
Epoch 15/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0299 - accuracy: 0.9867
Epoch 16/50
43/43 [======

43/43 [==============================] - 0s 1ms/step - loss: 0.0333 - accuracy: 0.9868
Epoch 12/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0101 - accuracy: 0.9949
Epoch 13/50
43/43 [==============================] - 0s 2ms/step - loss: 0.0323 - accuracy: 0.9919
Epoch 14/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0212 - accuracy: 0.9905
Epoch 15/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0184 - accuracy: 0.9927
Epoch 16/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0077 - accuracy: 0.9971
Epoch 17/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0032 - accuracy: 0.9985
Epoch 18/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0052 - accuracy: 0.9963
Epoch 19/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0235 - accuracy: 0.9956
Epoch 20/50
43/43 [==============================] - 0s 1ms/step - loss: 0.0318 - accuracy: 0.9905
Epoch 21/50
43/43 [===

In [13]:
final_model = MM_Sum(des_col, body_col, hid_layer, 1) # input type = integer
network = final_model.Multi_modal_Sum()
print(network)
concat_net = network[2]

concat_y, concat_sm_y, concat_cw_y,concat_smcw_y = [], [], [], []
output_con = train_model(X, y, concat_net, dict_y, dict_y, dict_y, dict_y)
# concatenate model training
concat_y = output_con.train_based()
#concat_sm_y = output_con.train_smote()
#concat_cw_y = output_con.train_weight()
#concat_smcw_y = output_con.train_sm_weight()

(<tensorflow.python.keras.engine.training.Model object at 0x0000024C4C027C08>, <tensorflow.python.keras.engine.training.Model object at 0x0000024C4C037648>, <tensorflow.python.keras.engine.training.Model object at 0x0000024C4C0441C8>)
TEST: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82]
Epoch 1/50
24/24 [==============================] - 0s 1ms/step - loss: 0.7033 - accuracy: 0.5409
Epoch 2/50
24/24 [==============================] - 0s 873us/step - loss: 0.5047 - accuracy: 0.6040
Epoch 3/50
24/24 [==============================] - 0s 914us/step - loss: 0.4367 - accuracy: 0.7034
Epoch 4/50
24/24 [==============================] - 0s 914us/step - loss: 0.4022 - accuracy: 0.7356
Epoch 5/50
24/24 [==============================] - 0s 831us/step - loss: 0.3433 - accuracy: 0.7906
Epoch 6

TEST: [249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266
 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284
 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302
 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320
 321 322 323 324 325 326 327 328 329 330 331]
Epoch 1/50
24/24 [==============================] - 0s 1ms/step - loss: 0.2273 - accuracy: 0.9570
Epoch 2/50
24/24 [==============================] - 0s 914us/step - loss: 0.0804 - accuracy: 0.9557
Epoch 3/50
24/24 [==============================] - 0s 914us/step - loss: 0.0850 - accuracy: 0.9678
Epoch 4/50
24/24 [==============================] - 0s 873us/step - loss: 0.0677 - accuracy: 0.9624
Epoch 5/50
24/24 [==============================] - 0s 873us/step - loss: 0.0538 - accuracy: 0.9826
Epoch 6/50
24/24 [==============================] - 0s 914us/step - loss: 0.0588 - accuracy: 0.9745
Epoch 7/50
24/24 [==============================] - 0s 831

Epoch 1/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0829 - accuracy: 0.9919
Epoch 2/50
24/24 [==============================] - 0s 873us/step - loss: 0.0558 - accuracy: 0.9906
Epoch 3/50
24/24 [==============================] - 0s 831us/step - loss: 0.0208 - accuracy: 0.9879
Epoch 4/50
24/24 [==============================] - 0s 831us/step - loss: 0.0131 - accuracy: 0.9933
Epoch 5/50
24/24 [==============================] - 0s 873us/step - loss: 0.0101 - accuracy: 0.9946
Epoch 6/50
24/24 [==============================] - 0s 873us/step - loss: 0.0151 - accuracy: 0.9946
Epoch 7/50
24/24 [==============================] - 0s 873us/step - loss: 0.0161 - accuracy: 0.9946
Epoch 8/50
24/24 [==============================] - 0s 914us/step - loss: 0.0095 - accuracy: 0.9960
Epoch 9/50
24/24 [==============================] - 0s 873us/step - loss: 0.0081 - accuracy: 0.9973
Epoch 10/50
24/24 [==============================] - 0s 873us/step - loss: 0.0166 - accuracy: 0.9973
E

24/24 [==============================] - 0s 873us/step - loss: 0.0208 - accuracy: 0.9933
Epoch 3/50
24/24 [==============================] - 0s 871us/step - loss: 0.0332 - accuracy: 0.9879
Epoch 4/50
24/24 [==============================] - 0s 873us/step - loss: 0.0212 - accuracy: 0.9853
Epoch 5/50
24/24 [==============================] - 0s 831us/step - loss: 0.0382 - accuracy: 0.9960
Epoch 6/50
24/24 [==============================] - 0s 873us/step - loss: 0.0248 - accuracy: 0.9879
Epoch 7/50
24/24 [==============================] - 0s 831us/step - loss: 0.0132 - accuracy: 0.9946
Epoch 8/50
24/24 [==============================] - 0s 831us/step - loss: 0.0285 - accuracy: 0.9879
Epoch 9/50
24/24 [==============================] - 0s 914us/step - loss: 0.0166 - accuracy: 0.9920
Epoch 10/50
24/24 [==============================] - 0s 831us/step - loss: 0.0093 - accuracy: 0.9960
Epoch 11/50
24/24 [==============================] - 0s 873us/step - loss: 0.0095 - accuracy: 0.9960
Epoch 12/

In [14]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc, f1_score, confusion_matrix, accuracy_score, matthews_corrcoef, accuracy_score
from imblearn.metrics import geometric_mean_score
import math

class Evaluation:
    def __init__(self, pred, y):
        self.y_pred = pred
        self.y = y
        
    def matrix(self):
        y_1 = self.y.reshape(self.y.shape[0],-1)
        y_pred = np.array(self.y_pred)
        y_pred = y_pred.reshape(y_pred.shape[0], -1)
        
        y_classify = []
        for i in range(len(self.y_pred)):
            if self.y_pred[i] >= 0.5:
                a = 1.
                y_classify.append(a)
            else:
                a = 0.
                y_classify.append(a)
        
        fpr,tpr,threshold = roc_curve(y_1 , y_pred, pos_label = 1)
        precision, recall, threshold = precision_recall_curve(y_1, y_pred, pos_label = 1)
        
        roc_auc = auc(fpr,tpr)
        auprc = auc(recall, precision)
        mean_precision = np.mean(precision)
        mean_recall = np.mean(recall)
        F1 = 2 * (mean_precision * mean_recall) / (mean_precision + mean_recall)
        # binary 
        accuracy = accuracy_score(y_1, y_classify)
        mcc = matthews_corrcoef(y_1, y_classify) 
        g_mean = geometric_mean_score(y_1, y_classify)
        confusion = confusion_matrix(y_1, y_classify)
        print(confusion.ravel())
        tn, fp, fn, tp = confusion.ravel()
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)
        ppv = tp / (tp + fp)
        fnr = fn / (fn + tp)
        fpr = fp / (fp + tn)

        confu_precision = ppv 
        confu_recall = tpr # sensitivity
        confu_f1 = 2 * ((ppv * tpr) / (ppv + tpr))
        confu_accuracy = (tp + tn) / (tp + tn + fp + fn)
        confu_mcc = ((tp * tn)-(fp-fn))/ math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))
        confu_g_mean = math.sqrt(tpr * tnr)
        Optimized_precision = (confu_accuracy - abs(tnr-tpr)) / (tnr + tpr)

        print('공통 \nAUC :',roc_auc) # pb
        print("AUPRC :", auprc) # pb
        print("Optimized precision :", Optimized_precision)

        print("\nfunction 사용\nAccuracy :", accuracy) #pb
        print("Precision(pb) :",mean_precision )
        print("Recall(pb) :", mean_recall) # pb
        print("F1 score(pb) :", F1) #pb

        print("MCC :", mcc)
        print("G-mean :", g_mean)


        print("\nConfusion_matrix 사용 \n", confusion)
        print("Accuracy :", confu_accuracy)
        print("Precision :", confu_precision) 
        print("Recall :", confu_recall) 
        print("F1 score :", confu_f1) 

        print("MCC :", confu_mcc)
        print("G-mean :", confu_g_mean)
        
        return roc_auc, auprc, Optimized_precision, accuracy, mean_precision, mean_recall, F1, mcc, g_mean, confu_accuracy, confu_precision, confu_recall, confu_f1, confu_mcc, confu_g_mean;

In [46]:
sum_y_result = Evaluation(predict_change(sum_y), y)
#sum_sm_y_result = Evaluation(sum_sm_y, y)
#sum_cw_y_result = Evaluation(sum_cw_y, y)
#sum_smcw_y_result = Evaluation(sum_smcw_y, y)

print("#sum model 결과 \n")
print("\n-----based model-----\n")
print(sum_y_result.matrix())

print("\n-----smote model-----\n")
#print(sum_sm_y_result.matrix())

print("\n-----class weight model-----\n")
#print(sum_cw_y_result.matrix())

print("\n-----smote + class weight model-----\n")
#print(sum_smcw_y_result.matrix())

#sum model 결과 


-----based model-----

[ 39  32  14 743]
공통 
AUC : 0.9264331032429717
AUPRC : 0.9917135017424152
Optimized precision : 0.3346183037040638

function 사용
Accuracy : 0.9444444444444444
Precision(pb) : 0.9867389924860067
Recall(pb) : 0.648725004902584
F1 score(pb) : 0.7828020167489639
MCC : 0.6071844613303631
G-mean : 0.7342595372996048

Confusion_matrix 사용 
 [[ 39  32]
 [ 14 743]]
Accuracy : 0.9444444444444444
Precision : 0.9587096774193549
Recall : 0.9815059445178336
F1 score : 0.9699738903394256
MCC : 0.6163361777723013
G-mean : 0.7342595372996048
(0.9264331032429717, 0.9917135017424152, 0.3346183037040638, 0.9444444444444444, 0.9867389924860067, 0.648725004902584, 0.7828020167489639, 0.6071844613303631, 0.7342595372996048, 0.9444444444444444, 0.9587096774193549, 0.9815059445178336, 0.9699738903394256, 0.6163361777723013, 0.7342595372996048)

-----smote model-----


-----class weight model-----


-----smote + class weight model-----



In [121]:
#multi_y_result = Evaluation(multiply_y, y)
#multi_sm_y_result = Evaluation(multiply_sm_y, y)
#multi_cw_y_result = Evaluation(multiply_cw_y, y)
multi_smcw_y_result = Evaluation(multiply_smcw_y, y)

print("#multiply model 결과 \n")
print("\n-----based model-----\n")
#print(multi_y_result.matrix())

print("\n-----smote model-----\n")
#print(multi_sm_y_result.matrix())

print("\n-----class weight model-----\n")
#print(multi_cw_y_result.matrix())

print("\n-----smote + class weight model-----\n")
print(multi_smcw_y_result.matrix())

#multiply model 결과 


-----based model-----


-----smote model-----


-----class weight model-----


-----smote + class weight model-----

공통 
AUC : 0.960081492920535
AUPRC : 0.9953698274500926
Optimized precision : 0.4772259601631522

function 사용
Accuracy : 0.9227053140096618
Precision(pb) : 0.9910013621523781
Recall(pb) : 0.6151879445595295
F1 score(pb) : 0.759128564099655
MCC : 0.6378386403319343
G-mean : 0.8932344529682904

Confusion_matrix 사용 
 [[ 61  10]
 [ 54 703]]
Accuracy : 0.9227053140096618
Precision : 0.9859747545582047
Recall : 0.9286657859973579
F1 score : 0.9564625850340135
MCC : 0.6466357913593496
G-mean : 0.8932344529682904
None


c:\users\user\anaconda3\envs\gpu\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [17]:
#concat_y_result = Evaluation(concat_y, y)
#concat_sm_y_result = Evaluation(concat_sm_y, y)
concat_cw_y_result = Evaluation(concat_cw_y, y)
#concat_smcw_y_result = Evaluation(concat_smcw_y, y)

print("#concatenate model 결과 \n")
print("\n-----based model-----\n")
#print(concat_y_result.matrix())

print("\n-----smote model-----\n")
#print(concat_sm_y_result.matrix())

print("\n-----class weight model-----\n")
print(concat_cw_y_result.matrix())

print("\n-----smote + class weight model-----\n")
#print(concat_smcw_y_result.matrix())

#concatenate model 결과 


-----based model-----


-----smote model-----


-----class weight model-----

공통 
AUC : 0.9392710290806928
AUPRC : 0.9931842493052667
Optimized precision : 0.42550210482341977

function 사용
Accuracy : 0.927536231884058
Precision(pb) : 0.9877802610019468
Recall(pb) : 0.6397568069790704
F1 score(pb) : 0.7765588363028656
MCC : 0.6014105159209538
G-mean : 0.8323022503439401

Confusion_matrix 사용 
 [[ 52  19]
 [ 41 716]]
Accuracy : 0.927536231884058
Precision : 0.9741496598639455
Recall : 0.9458388375165125
F1 score : 0.9597855227882037
MCC : 0.6146256099667851
G-mean : 0.83230225034394
None

-----smote + class weight model-----



c:\users\user\anaconda3\envs\gpu\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [141]:
sum_y_set = []

for i in range(20):
    final_model = MM_Sum(des_col, body_col, hid_layer, 1) # input type = integer
    network = final_model.Multi_modal_Sum()
    print(network)
    sum_net = network[0]

    sum_y, sum_sm_y, sum_cw_y, sum_smcw_y = [], [], [], []
    output_sum = train_model(X, y, sum_net, dict_y, dict_y, dict_y, dict_y)
    
    # sum model training
    sum_y = output_sum.train_based()
    #sum_sm_y = output_sum.train_smote()
    #sum_cw_y = output_sum.train_weight()
    #sum_smcw_y = output_sum.train_sm_weight()

    
    sum_y_result = Evaluation(predict_change(sum_y), y)
    #sum_sm_y_result = Evaluation(sum_sm_y, y)
    #sum_cw_y_result = Evaluation(predict_change(sum_cw_y), y)
    #sum_smcw_y_result = Evaluation(predict_change(sum_smcw_y), y)
    
    sum_y_set = np.append(sum_y_set, sum_y_result.matrix())
    print(len(sum_y_set))
    
sum_y_np = np.array(sum_y_set)

sum_y_np = sum_y_np.reshape(-1,15)
sum_y_np.shape

(<tensorflow.python.keras.engine.training.Model object at 0x0000025D213EB648>, <tensorflow.python.keras.engine.training.Model object at 0x0000025D3DC2D988>, <tensorflow.python.keras.engine.training.Model object at 0x0000025D07743808>)
TEST: [  1  11  12  34  36  49  56  71  93 112 119 125 126 140 145 152 157 183
 190 194 198 205 209 210 211 214 233 237 257 264 272 298 303 323 328 340
 354 368 373 380 399 403 409 412 422 432 434 447 456 461 464 468 474 476
 491 496 506 518 520 523 528 532 538 547 553 556 589 611 624 629 645 658
 665 700 702 708 735 757 766 774 799 800 825]
Epoch 1/50
24/24 [==============================] - 0s 1ms/step - loss: 0.8344 - accuracy: 0.5826
Epoch 2/50
24/24 [==============================] - 0s 1ms/step - loss: 0.5132 - accuracy: 0.7651
Epoch 3/50
24/24 [==============================] - 0s 1ms/step - loss: 0.3848 - accuracy: 0.8859
Epoch 4/50
24/24 [==============================] - 0s 1ms/step - loss: 0.3113 - accuracy: 0.9034
Epoch 5/50
24/24 [===========

Epoch 1/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0398 - accuracy: 0.9852
Epoch 2/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0315 - accuracy: 0.9866
Epoch 3/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0266 - accuracy: 0.9893
Epoch 4/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0247 - accuracy: 0.9906
Epoch 5/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0197 - accuracy: 0.9960
Epoch 6/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0096 - accuracy: 0.9973
Epoch 7/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0089 - accuracy: 0.9973
Epoch 8/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0098 - accuracy: 0.9960
Epoch 9/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0121 - accuracy: 0.9960
Epoch 10/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0255 - accuracy: 0.9933
Epoch 11/50
24/24 [

24/24 [==============================] - 0s 1ms/step - loss: 0.0077 - accuracy: 0.9973
Epoch 7/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0054 - accuracy: 0.9973
Epoch 8/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0074 - accuracy: 0.9973
Epoch 9/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0080 - accuracy: 0.9973
Epoch 10/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0116 - accuracy: 0.9973
Epoch 11/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0203 - accuracy: 0.9906
Epoch 12/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0101 - accuracy: 0.9973
Epoch 13/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0094 - accuracy: 0.9973
Epoch 14/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0083 - accuracy: 0.9973
Epoch 15/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0058 - accuracy: 0.9987
Epoch 16/50
24/24 [======

24/24 [==============================] - 0s 2ms/step - loss: 0.0126 - accuracy: 0.9960
Epoch 12/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 13/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0100 - accuracy: 0.9987
Epoch 14/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0045 - accuracy: 0.9987
Epoch 15/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0022 - accuracy: 0.9987
Epoch 16/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0059 - accuracy: 0.9973
Epoch 17/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0160 - accuracy: 0.9960
Epoch 18/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0110 - accuracy: 0.9973
Epoch 19/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 20/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 21/50
24/24 [===

24/24 [==============================] - 0s 1ms/step - loss: 0.0359 - accuracy: 0.9906
Epoch 9/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0599 - accuracy: 0.9826
Epoch 10/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0272 - accuracy: 0.9879
Epoch 11/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0172 - accuracy: 0.9946
Epoch 12/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0249 - accuracy: 0.9946
Epoch 13/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0270 - accuracy: 0.9906
Epoch 14/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0348 - accuracy: 0.9906
Epoch 15/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0353 - accuracy: 0.9893
Epoch 16/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0512 - accuracy: 0.9745
Epoch 17/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0648 - accuracy: 0.9772
Epoch 18/50
24/24 [====

...

24/24 [==============================] - 0s 1ms/step - loss: 0.0056 - accuracy: 0.9973
Epoch 17/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0062 - accuracy: 0.9946
Epoch 18/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 19/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0042 - accuracy: 0.9987
Epoch 20/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0030 - accuracy: 0.9987
Epoch 21/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0072 - accuracy: 0.9987
Epoch 22/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0103 - accuracy: 0.9960
Epoch 23/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0149 - accuracy: 0.9933
Epoch 24/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0088 - accuracy: 0.9946
Epoch 25/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0091 - accuracy: 0.9960
Epoch 26/50
24/24 [===

24/24 [==============================] - 0s 1ms/step - loss: 0.0523 - accuracy: 0.9758
Epoch 14/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0329 - accuracy: 0.9879
Epoch 15/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0275 - accuracy: 0.9879
Epoch 16/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0204 - accuracy: 0.9946
Epoch 17/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0199 - accuracy: 0.9933
Epoch 18/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0261 - accuracy: 0.9879
Epoch 19/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0164 - accuracy: 0.9933
Epoch 20/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0208 - accuracy: 0.9906
Epoch 21/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0344 - accuracy: 0.9839
Epoch 22/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0155 - accuracy: 0.9946
Epoch 23/50
24/24 [===

24/24 [==============================] - 0s 2ms/step - loss: 0.0148 - accuracy: 0.9946
Epoch 19/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 20/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 21/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0056 - accuracy: 0.9987
Epoch 22/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0120 - accuracy: 0.9973
Epoch 23/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0071 - accuracy: 0.9960
Epoch 24/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0070 - accuracy: 0.9987
Epoch 25/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0043 - accuracy: 0.9987
Epoch 26/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0097 - accuracy: 0.9973
Epoch 27/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0062 - accuracy: 0.9987
Epoch 28/50
24/24 [===

24/24 [==============================] - 0s 1ms/step - loss: 0.0054 - accuracy: 0.9987
Epoch 24/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0118 - accuracy: 0.9920
Epoch 25/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0037 - accuracy: 0.9987
Epoch 26/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0124 - accuracy: 0.9946
Epoch 27/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0090 - accuracy: 0.9973
Epoch 28/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0156 - accuracy: 0.9933
Epoch 29/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0240 - accuracy: 0.9920
Epoch 30/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0071 - accuracy: 0.9973
Epoch 31/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0048 - accuracy: 0.9987
Epoch 32/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0058 - accuracy: 0.9973
Epoch 33/50
24/24 [===

(20, 15)

In [18]:
multi_y_set = []

for i in range(20):
    final_model = MM_Sum(des_col, body_col, hid_layer, 1) # input type = integer
    network = final_model.Multi_modal_Sum()
    print(network)
    multiply_net = network[1]

    multiply_y, multiply_sm_y, multiply_cw_y, multiply_smcw_y =[], [], [], []
    output_mul = train_model(X, y, multiply_net, dict_y, dict_y, dict_y, dict_y)

    # multiple model training
    multiply_y = output_mul.train_based()
    #multiply_sm_y = output_mul.train_smote()
    #multiply_cw_y = output_mul.train_weight()
    #multiply_smcw_y = output_mul.train_sm_weight()

    multi_y_result = Evaluation(predict_change(multiply_y), y)
    #multi_sm_y_result = Evaluation(multiply_sm_y, y)
    #multi_cw_y_result = Evaluation(predict_change(multiply_cw_y), y)
    #multi_smcw_y_result = Evaluation(predict_change(multiply_smcw_y), y)
    
    multi_y_set = np.append(multi_y_set, multi_y_result.matrix())
    print(len(multi_y_set))
    
multi_y_np = np.array(multi_y_set)

multi_y_np = multi_y_np.reshape(-1,15)
multi_y_np.shape

(<tensorflow.python.keras.engine.training.Model object at 0x000002AA5B7F1DC8>, <tensorflow.python.keras.engine.training.Model object at 0x000002AA5B8078C8>, <tensorflow.python.keras.engine.training.Model object at 0x000002AA5B8020C8>)
TEST: [  1  11  12  34  36  49  56  71  93 112 119 125 126 140 145 152 157 183
 190 194 198 205 209 210 211 214 233 237 257 264 272 298 303 323 328 340
 354 368 373 380 399 403 409 412 422 432 434 447 456 461 464 468 474 476
 491 496 506 518 520 523 528 532 538 547 553 556 589 611 624 629 645 658
 665 700 702 708 735 757 766 774 799 800 825]
Epoch 1/50
24/24 [==============================] - 0s 1ms/step - loss: 0.7768 - accuracy: 0.5745
Epoch 2/50
24/24 [==============================] - 0s 997us/step - loss: 0.5265 - accuracy: 0.7436
Epoch 3/50
24/24 [==============================] - 0s 1ms/step - loss: 0.3967 - accuracy: 0.8336
Epoch 4/50
24/24 [==============================] - 0s 997us/step - loss: 0.3550 - accuracy: 0.8483
Epoch 5/50
24/24 [=======

24/24 [==============================] - 0s 1ms/step - loss: 0.0155 - accuracy: 0.9960
Epoch 25/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0177 - accuracy: 0.9946
Epoch 26/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0167 - accuracy: 0.9973
Epoch 27/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0140 - accuracy: 0.9960
Epoch 28/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0109 - accuracy: 0.9973
Epoch 29/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0164 - accuracy: 0.9946
Epoch 30/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0136 - accuracy: 0.9973
Epoch 31/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0190 - accuracy: 0.9946
Epoch 32/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0087 - accuracy: 1.0000
Epoch 33/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0207 - accuracy: 0.9933
Epoch 34/50
24/24 [===

24/24 [==============================] - 0s 1ms/step - loss: 0.0346 - accuracy: 0.9879
Epoch 2/50
24/24 [==============================] - 0s 651us/step - loss: 0.0180 - accuracy: 0.9933
Epoch 3/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0118 - accuracy: 0.9946
Epoch 4/50
24/24 [==============================] - 0s 651us/step - loss: 0.0130 - accuracy: 0.9960
Epoch 5/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0078 - accuracy: 0.9973
Epoch 6/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0089 - accuracy: 0.9987
Epoch 7/50
24/24 [==============================] - 0s 650us/step - loss: 0.0080 - accuracy: 0.9973
Epoch 8/50
24/24 [==============================] - 0s 650us/step - loss: 0.0109 - accuracy: 0.9960
Epoch 9/50
24/24 [==============================] - 0s 922us/step - loss: 0.0230 - accuracy: 0.9919
Epoch 10/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0251 - accuracy: 0.9933
Epoch 11/50
24/24 [=

Epoch 4/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 5/50
24/24 [==============================] - 0s 646us/step - loss: 0.0064 - accuracy: 0.9960
Epoch 6/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0114 - accuracy: 0.9960
Epoch 7/50
24/24 [==============================] - 0s 651us/step - loss: 0.0082 - accuracy: 0.9960
Epoch 8/50
24/24 [==============================] - 0s 651us/step - loss: 0.0136 - accuracy: 0.9946
Epoch 9/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0059 - accuracy: 0.9987
Epoch 10/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0123 - accuracy: 0.9946
Epoch 11/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0154 - accuracy: 0.9946
Epoch 12/50
24/24 [==============================] - 0s 507us/step - loss: 0.0181 - accuracy: 0.9933
Epoch 13/50
24/24 [==============================] - 0s 651us/step - loss: 0.0049 - accuracy: 0.9973
Epoch 

24/24 [==============================] - 0s 923us/step - loss: 0.0055 - accuracy: 0.9987
Epoch 33/50
24/24 [==============================] - 0s 652us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 34/50
24/24 [==============================] - 0s 651us/step - loss: 0.0098 - accuracy: 0.9987
Epoch 35/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0190 - accuracy: 0.9919
Epoch 36/50
24/24 [==============================] - 0s 922us/step - loss: 0.0312 - accuracy: 0.9919
Epoch 37/50
24/24 [==============================] - 0s 651us/step - loss: 0.0246 - accuracy: 0.9946
Epoch 38/50
24/24 [==============================] - 0s 651us/step - loss: 0.0153 - accuracy: 0.9946
Epoch 39/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0094 - accuracy: 0.9960
Epoch 40/50
24/24 [==============================] - 0s 738us/step - loss: 0.0179 - accuracy: 0.9973
Epoch 41/50
24/24 [==============================] - 0s 651us/step - loss: 0.0088 - accuracy: 0.9960
Epoch 

...

24/24 [==============================] - 0s 1ms/step - loss: 0.0260 - accuracy: 0.9919
Epoch 23/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0295 - accuracy: 0.9893
Epoch 24/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0265 - accuracy: 0.9893
Epoch 25/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0139 - accuracy: 0.9973
Epoch 26/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0220 - accuracy: 0.9933
Epoch 27/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0247 - accuracy: 0.9906
Epoch 28/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0260 - accuracy: 0.9893
Epoch 29/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0250 - accuracy: 0.9906
Epoch 30/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0113 - accuracy: 0.9973
Epoch 31/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0081 - accuracy: 0.9973
Epoch 32/50
24/24 [===

Epoch 1/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0520 - accuracy: 0.9893
Epoch 2/50
24/24 [==============================] - 0s 651us/step - loss: 0.0329 - accuracy: 0.9906
Epoch 3/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0238 - accuracy: 0.9946
Epoch 4/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0188 - accuracy: 0.9960
Epoch 5/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0145 - accuracy: 0.9960
Epoch 6/50
24/24 [==============================] - 0s 651us/step - loss: 0.0075 - accuracy: 1.0000
Epoch 7/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0061 - accuracy: 0.9987
Epoch 8/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0150 - accuracy: 0.9960
Epoch 9/50
24/24 [==============================] - 0s 651us/step - loss: 0.0140 - accuracy: 0.9946
Epoch 10/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0150 - accuracy: 0.9946
Epoch 11/50
2

Epoch 30/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0041 - accuracy: 1.0000
Epoch 31/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0043 - accuracy: 0.9987
Epoch 32/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0041 - accuracy: 0.9987
Epoch 33/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0157 - accuracy: 0.9946
Epoch 34/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0124 - accuracy: 0.9933
Epoch 35/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0065 - accuracy: 0.9987
Epoch 36/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0092 - accuracy: 0.9973
Epoch 37/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0071 - accuracy: 0.9973
Epoch 38/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0092 - accuracy: 0.9987
Epoch 39/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0085 - accuracy: 0.9973
Epoch 40/5

24/24 [==============================] - 0s 828us/step - loss: 0.0025 - accuracy: 0.9987
Epoch 35/50
24/24 [==============================] - 0s 1ms/step - loss: 6.9290e-04 - accuracy: 1.0000
Epoch 36/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0039 - accuracy: 0.9973
Epoch 37/50
24/24 [==============================] - 0s 697us/step - loss: 0.0058 - accuracy: 0.9960
Epoch 38/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0042 - accuracy: 0.9987
Epoch 39/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0046 - accuracy: 0.9987
Epoch 40/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 41/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 42/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 43/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0029 - accuracy: 0.9987
Epoch 44/50
24

(20, 15)

In [24]:
concat_y_set = []

for i in range(20):
    final_model = MM_Sum(des_col, body_col, hid_layer, 1) # input type = integer
    network = final_model.Multi_modal_Sum()
    print(network)
    concat_net = network[2]

    concat_y, concat_sm_y, concat_cw_y,concat_smcw_y = [], [], [], []
    output_con = train_model(X, y, concat_net, dict_y, dict_y, dict_y, dict_y)
    
    # concatenate model training
    concat_y = output_con.train_based()
    #concat_sm_y = output_con.train_smote()
    #concat_cw_y = output_con.train_weight()
    #concat_smcw_y = output_con.train_sm_weight()

    concat_y_result = Evaluation(predict_change(concat_y), y)
    #concat_sm_y_result = Evaluation(concat_sm_y, y)
    #concat_cw_y_result = Evaluation(predict_change(concat_cw_y), y)
    #concat_smcw_y_result = Evaluation(predict_change(concat_smcw_y), y)
    

    
    concat_y_set = np.append(concat_y_set, concat_y_result.matrix())
    print(len(concat_y_set))
    
concat_y_np = np.array(concat_y_set)

concat_y_np = concat_y_np.reshape(-1,15)
concat_y_np.shape

(<tensorflow.python.keras.engine.training.Model object at 0x000002AA8E741DC8>, <tensorflow.python.keras.engine.training.Model object at 0x000002AA8E74E948>, <tensorflow.python.keras.engine.training.Model object at 0x000002AA8E75D308>)
TEST: [  1  11  12  34  36  49  56  71  93 112 119 125 126 140 145 152 157 183
 190 194 198 205 209 210 211 214 233 237 257 264 272 298 303 323 328 340
 354 368 373 380 399 403 409 412 422 432 434 447 456 461 464 468 474 476
 491 496 506 518 520 523 528 532 538 547 553 556 589 611 624 629 645 658
 665 700 702 708 735 757 766 774 799 800 825]
Epoch 1/50
24/24 [==============================] - 0s 2ms/step - loss: 0.7203 - accuracy: 0.5772
Epoch 2/50
24/24 [==============================] - 0s 2ms/step - loss: 0.5261 - accuracy: 0.7691
Epoch 3/50
24/24 [==============================] - 0s 2ms/step - loss: 0.4220 - accuracy: 0.8698
Epoch 4/50
24/24 [==============================] - 0s 2ms/step - loss: 0.3457 - accuracy: 0.9208
Epoch 5/50
24/24 [===========

24/24 [==============================] - 0s 2ms/step - loss: 0.0267 - accuracy: 0.9879
Epoch 2/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0184 - accuracy: 0.9960
Epoch 3/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0118 - accuracy: 0.9960
Epoch 4/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0296 - accuracy: 0.9852
Epoch 5/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0185 - accuracy: 0.9879
Epoch 6/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0486 - accuracy: 0.9852
Epoch 7/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0266 - accuracy: 0.9919
Epoch 8/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0249 - accuracy: 0.9933
Epoch 9/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0149 - accuracy: 0.9946
Epoch 10/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0125 - accuracy: 0.9960
Epoch 11/50
24/24 [===========

24/24 [==============================] - 0s 2ms/step - loss: 0.0125 - accuracy: 0.9946
Epoch 7/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0087 - accuracy: 0.9973
Epoch 8/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0104 - accuracy: 0.9960
Epoch 9/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0071 - accuracy: 0.9987
Epoch 10/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0066 - accuracy: 0.9987
Epoch 11/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0205 - accuracy: 0.9933
Epoch 12/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0128 - accuracy: 0.9960
Epoch 13/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0143 - accuracy: 0.9946
Epoch 14/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0115 - accuracy: 0.9960
Epoch 15/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0167 - accuracy: 0.9973
Epoch 16/50
24/24 [======

24/24 [==============================] - 0s 2ms/step - loss: 0.0138 - accuracy: 0.9960
Epoch 11/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0252 - accuracy: 0.9920
Epoch 12/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0193 - accuracy: 0.9946
Epoch 13/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0054 - accuracy: 0.9987
Epoch 14/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0106 - accuracy: 0.9960
Epoch 15/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0108 - accuracy: 0.9946
Epoch 16/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0078 - accuracy: 0.9973
Epoch 17/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0055 - accuracy: 0.9987
Epoch 18/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0065 - accuracy: 0.9987
Epoch 19/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0123 - accuracy: 0.9946
Epoch 20/50
24/24 [===

24/24 [==============================] - 0s 2ms/step - loss: 0.0182 - accuracy: 0.9933
Epoch 8/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0221 - accuracy: 0.9919
Epoch 9/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0230 - accuracy: 0.9933
Epoch 10/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0231 - accuracy: 0.9879
Epoch 11/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0427 - accuracy: 0.9839
Epoch 12/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0350 - accuracy: 0.9772
Epoch 13/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0310 - accuracy: 0.9852
Epoch 14/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0348 - accuracy: 0.9852
Epoch 15/50
24/24 [==============================] - 0s 1ms/step - loss: 0.0398 - accuracy: 0.9866
Epoch 16/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0286 - accuracy: 0.9933
Epoch 17/50
24/24 [=====

...

24/24 [==============================] - 0s 2ms/step - loss: 0.0140 - accuracy: 0.9946
Epoch 2/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0074 - accuracy: 0.9987
Epoch 3/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0126 - accuracy: 0.9960
Epoch 4/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0073 - accuracy: 0.9960
Epoch 5/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0254 - accuracy: 0.9893
Epoch 6/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0101 - accuracy: 0.9973
Epoch 7/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0053 - accuracy: 0.9960
Epoch 8/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0049 - accuracy: 0.9987
Epoch 9/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0105 - accuracy: 0.9973
Epoch 10/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0129 - accuracy: 0.9960
Epoch 11/50
24/24 [===========

Epoch 1/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0390 - accuracy: 0.9839
Epoch 2/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0372 - accuracy: 0.9866
Epoch 3/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0386 - accuracy: 0.9866
Epoch 4/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0295 - accuracy: 0.9839
Epoch 5/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0271 - accuracy: 0.9919
Epoch 6/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0198 - accuracy: 0.9946
Epoch 7/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0259 - accuracy: 0.9919
Epoch 8/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0378 - accuracy: 0.9866
Epoch 9/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0470 - accuracy: 0.9852
Epoch 10/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0343 - accuracy: 0.9839
Epoch 11/50
24/24 [

24/24 [==============================] - 0s 2ms/step - loss: 0.0123 - accuracy: 0.9960
Epoch 6/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0165 - accuracy: 0.9906
Epoch 7/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0121 - accuracy: 0.9933
Epoch 8/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0192 - accuracy: 0.9946
Epoch 9/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0131 - accuracy: 0.9960
Epoch 10/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0125 - accuracy: 0.9946
Epoch 11/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0141 - accuracy: 0.9946
Epoch 12/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0081 - accuracy: 0.9973
Epoch 13/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0046 - accuracy: 0.9987
Epoch 14/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 15/50
24/24 [=======

24/24 [==============================] - 0s 2ms/step - loss: 0.0077 - accuracy: 0.9987
Epoch 11/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0060 - accuracy: 0.9973
Epoch 12/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0126 - accuracy: 0.9960
Epoch 13/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0117 - accuracy: 0.9960
Epoch 14/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0073 - accuracy: 0.9987
Epoch 15/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0091 - accuracy: 0.9960
Epoch 16/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 17/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0049 - accuracy: 0.9987
Epoch 18/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0095 - accuracy: 0.9960
Epoch 19/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0052 - accuracy: 0.9987
Epoch 20/50
24/24 [===

(20, 15)

In [16]:
def result(data):
    auc = data[:,0]
    auprc = data[:,1]
    op = data[:,2]
    acc = data[:,3]
    precision = data[:,4]
    recall = data[:,5]
    f1 = data[:,6]
    mcc = data[:,7]
    g_mean = data[:,8]
    confu_acc = data[:,9]
    confu_precision = data[:,10]
    confu_recall = data[:,11]
    confu_f1 = data[:,12]
    confu_mcc = data[:,13]
    confu_g_mean = data[:,14]

    print('공통 \nAUC :\n',"[평균 :",format(auc.mean(), ".5f"), "] [표준편차 :",format(auc.std(), ".5f"),"]") # pb
    print("AUPRC :\n", "[평균 :",format(auprc.mean(), ".5f"), "] [표준편차 :",format(auprc.std(), ".5f"),"]")
    print("Optimized precision :\n","[평균 :",format(op.mean(), ".5f"), "] [표준편차 :",format(op.std(), ".5f"),"]")

    print("\nfunction 사용\nAccuracy :\n","[평균 :",format(acc.mean(), ".5f"), "] [표준편차 :",format(acc.std(), ".5f"),"]")
    print("Precision(pb) :\n", "[평균 :",format(precision.mean(), ".5f"), "] [표준편차 :",format(precision.std(), ".5f"),"]")
    print("Recall(pb) :\n", "[평균 :",format(recall.mean(), ".5f"), "] [표준편차 :",format(recall.std(), ".5f"),"]")
    print("F1 score(pb) :\n","[평균 :",format(f1.mean(), ".5f"), "] [표준편차 :",format(f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(mcc.mean(), ".5f"), "] [표준편차 :",format(mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(g_mean.mean(), ".5f"), "] [표준편차 :",format(g_mean.std(), ".5f"),"]")
    
    print("\nConfusion matrix 사용 \nAccuracy :\n", "[평균 :",format(confu_acc.mean(), ".5f"), "] [표준편차 :",format(confu_acc.std(), ".5f"),"]")
    print("Precision :\n", "[평균 :",format(confu_precision.mean(), ".5f"), "] [표준편차 :",format(confu_precision.std(), ".5f"),"]")
    print("Recall :\n", "[평균 :",format(confu_recall.mean(), ".5f"), "] [표준편차 :",format(confu_recall.std(), ".5f"),"]")
    print("F1 score :\n", "[평균 :",format(confu_f1.mean(), ".5f"), "] [표준편차 :",format(confu_f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(confu_mcc.mean(), ".5f"), "] [표준편차 :",format(confu_mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(confu_g_mean.mean(), ".5f"), "] [표준편차 :",format(confu_g_mean.std(), ".5f"),"]")
    

In [25]:
result(concat_y_np)

공통 
AUC :
 [평균 : 0.96214 ] [표준편차 : 0.00821 ]
AUPRC :
 [평균 : 0.99577 ] [표준편차 : 0.00111 ]
Optimized precision :
 [평균 : 0.39120 ] [표준편차 : 0.01881 ]

function 사용
Accuracy :
 [평균 : 0.95743 ] [표준편차 : 0.00588 ]
Precision(pb) :
 [평균 : 0.99137 ] [표준편차 : 0.00187 ]
Recall(pb) :
 [평균 : 0.69711 ] [표준편차 : 0.01149 ]
F1 score(pb) :
 [평균 : 0.81854 ] [표준편차 : 0.00773 ]
MCC :
 [평균 : 0.71187 ] [표준편차 : 0.04074 ]
G-mean :
 [평균 : 0.81549 ] [표준편차 : 0.02748 ]

Confusion matrix 사용 
Accuracy :
 [평균 : 0.95743 ] [표준편차 : 0.00588 ]
Precision :
 [평균 : 0.97012 ] [표준편차 : 0.00409 ]
Recall :
 [평균 : 0.98375 ] [표준편차 : 0.00448 ]
F1 score :
 [평균 : 0.97688 ] [표준편차 : 0.00320 ]
MCC :
 [평균 : 0.71732 ] [표준편차 : 0.03904 ]
G-mean :
 [평균 : 0.81549 ] [표준편차 : 0.02748 ]
